In [ ]:
from typing import Union
from sqlmodel import SQLModel, Session, create_engine, select
import random
import simpy
from faker import Faker
import utilities
import models
import os

fake = Faker()

Faker.seed(42)
random.seed(42)

SIMULATION_DURATION_IN_YEARS = 0
SIMULATION_DURATION_IN_MONTHS = 1
SIMULATION_DURATION_IN_MINUTES = (60 * 24) * (7 * 4) * SIMULATION_DURATION_IN_MONTHS

NUMBER_OF_PRACTITIONERS = 50
NUMBER_OF_PATIENTS = 1500 * NUMBER_OF_PRACTITIONERS

EVENT_TYPE_WEIGHTS = {
    "Appointment": 1.0 / 3.0,
    "Encounter": 1.0 / 3.0,
    "Observation": 1.0 / 3.0,
}

APPOINTMENT_VISIT_DURATIONS = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
APPOINTMENT_VISIT_DURATION = lambda: random.choice(APPOINTMENT_VISIT_DURATIONS)

APPOINTMENT_CANCEL_PROBABILITY = 0.10
APPOINTMENT_NOSHOW_PROBABILITY = 0.10

OBSERVATIONS_DURING_ENCOUNTER_PROBABILITY = 1.0 / 3.0
OBSERVATIONS_DURING_APPOINTMENT_PROBABILITY = 1.0 / 3.0

OBSERVATION_CODES = {
    0: ("8310-5", "Body Temperature"),
    1: ("8867-4", "Heart Rate"),
    2: ("9279-1", "Respiratory Rate"),
    3: ("8480-6", "Blood Pressure"),
    4: ("2345-7", "Blood Glucose"),
}
OBSERVATIONS_MAX = 5

BTG_ACCESS_PROBABILITY = 0.15
STANDALONE_BTG_ACCESS_PROBABILITY = 0.15
STANDALONE_NORMAL_ACCESS_PROBABILITY = 0.10

lambda_1 = 1
lambda_2 = 7
lambda_3 = 31 * 1
lambda_4 = 31 * 3
lambda_multiplier = 60 * 24

def sample_cooldown_time():
    if random.random() < 0.75:
        return random.randint(lambda_1, lambda_2)
    else:
        return random.randint(lambda_3, lambda_4)

PATIENT_SCHEDULING_COOLDOWN_IN_DAYS = lambda: sample_cooldown_time()
PATIENT_SCHEDULING_COOLDOWN_IN_MINUTES = (
    lambda: lambda_multiplier * PATIENT_SCHEDULING_COOLDOWN_IN_DAYS()
)

if SIMULATION_DURATION_IN_MINUTES < lambda_4:
    raise ValueError(
        "The simulation duration < max patient arrival time."
        + " Patients may arrive after the simulation has concluded."
    )

last_patient_activity: dict = {}
active_appointments: set[tuple[str, str]] = set()

PATIENT_DISCHARGE_PROBABILITY = 0.10
PATIENT_ADMITTANCE_PROBABILITY = 0.10

PATIENT_TARGET_POPULATION = int(NUMBER_OF_PATIENTS * 1.00)
PATIENT_MIN_POPULATION = int(PATIENT_TARGET_POPULATION * 0.75)


def find_next_available_time(
    engine,
    requested_time: int,
    practitioner_object: models.Practitioner,
    appointment_duration: int,
) -> int:
    def fetch_busy_slots(session: Session) -> list[tuple[int, int]]:
        search_window_end = requested_time + 14 * 24 * 60

        appointments = session.exec(
            select(models.Appointment)
            .where(models.Appointment.practitioner_id == practitioner_object.id)
            .where(
                (
                    models.Appointment.status.in_(
                        [
                            models.AppointmentStatus.BOOKED,
                            models.AppointmentStatus.NOSHOW,
                        ]
                    )
                )
                & (models.Appointment.scheduled_start_time < search_window_end)
                & (
                    (
                        models.Appointment.scheduled_start_time
                        + models.Appointment.duration
                    )
                    > requested_time
                )
            )
        ).all()

        encounters = session.exec(
            select(models.Encounter).where(
                (models.Encounter.practitioner_id == practitioner_object.id)
                & (models.Encounter.actual_start_time < search_window_end)
                & (
                    (models.Encounter.actual_start_time + models.Encounter.duration)
                    > requested_time
                )
            )
        ).all()

        observations = session.exec(
            select(models.Observation).where(
                (models.Observation.practitioner_id == practitioner_object.id)
                & (models.Observation.timestamp < search_window_end)
                & ((models.Observation.timestamp + 1) > requested_time)
            )
        ).all()

        booked_slots = [
            (x.scheduled_start_time, x.scheduled_start_time + x.duration)
            for x in appointments
        ]
        encounter_slots = [
            (x.actual_start_time, x.actual_start_time + x.duration) for x in encounters
        ]
        observation_slots = [(x.timestamp, x.timestamp + 1) for x in observations]

        all_slots = booked_slots + encounter_slots + observation_slots
        return sorted(all_slots, key=lambda x: x[0])

    def is_within_working_hours(t_start: int, t_end: int) -> bool:
        day = (t_start // (24 * 60)) % 7
        minute_of_day_start = t_start % (24 * 60)
        minute_of_day_end = t_end % (24 * 60)

        if practitioner_object._work_schedule is None:
            raise ValueError("No work schedule defined for practitioner!")

        for work_start, work_end in practitioner_object._work_schedule.get(day, []):
            if work_start <= minute_of_day_start and minute_of_day_end <= work_end:
                return True
        return False

    with Session(engine) as session:
        busy_slots = fetch_busy_slots(session)
        current_time = max(requested_time, 0)

        for slot_start, slot_end in busy_slots:
            if current_time + appointment_duration <= slot_start:
                if is_within_working_hours(
                    current_time, current_time + appointment_duration
                ):
                    return current_time
            if slot_end > current_time:
                current_time = slot_end

        search_window_end = requested_time + 7 * 24 * 60
        while current_time + appointment_duration <= search_window_end:
            if is_within_working_hours(
                current_time, current_time + appointment_duration
            ):
                return current_time
            current_time += 1

        raise Exception("No available slot found in the next 7 days.")


def is_time_available(
    engine,
    environment,
    practitioner_object: models.Practitioner,
    start_time: int,
    duration: int,
) -> bool:
    end_time = start_time + duration

    with Session(engine) as session:
        conflicting_appointments = session.exec(
            select(models.Appointment)
            .where(models.Appointment.practitioner_id == practitioner_object.id)
            .where(models.Appointment.status == models.AppointmentStatus.BOOKED)
            .where(models.Appointment.scheduled_start_time < end_time)
            .where(
                models.Appointment.scheduled_start_time + models.Appointment.duration
                > start_time
            )
        ).first()

        if conflicting_appointments:
            return False

        conflicting_encounters = session.exec(
            select(models.Encounter)
            .where(models.Encounter.practitioner_id == practitioner_object.id)
            .where(models.Encounter.actual_start_time < end_time)
            .where(
                models.Encounter.actual_start_time + models.Encounter.duration
                > start_time
            )
        ).first()

        if conflicting_encounters:
            return False

        conflicting_observations = session.exec(
            select(models.Observation)
            .where(models.Observation.practitioner_id == practitioner_object.id)
            .where(models.Observation.timestamp >= start_time)
            .where(models.Observation.timestamp < end_time)
        ).first()

        if conflicting_observations:
            return False

    day = (start_time // (24 * 60)) % 7
    minute_of_day = start_time % (24 * 60)

    if practitioner_object._work_schedule is None:
        raise ValueError("Practitioner has no work schedule defined")

    for work_start, work_end in practitioner_object._work_schedule.get(day, []):
        if work_start <= minute_of_day <= work_end - duration:
            return True

    return False


def appointment(
    engine,
    environment,
    fhir_logger: models.FHIRLogger,
    practitioner_object: models.Practitioner,
    patient_object: models.Patient,
):
    appointment_duration = APPOINTMENT_VISIT_DURATION()
    key = (patient_object.id, practitioner_object.id)

    requested_time = environment.now
    scheduled_start_time = find_next_available_time(
        engine, requested_time, practitioner_object, appointment_duration
    )

    if scheduled_start_time is None:
        print(
            f"[{environment.now:>4}] No available time found for {patient_object.id} with {practitioner_object.id}"
        )
        return None

    appointment_id = fhir_logger.log_appointment(
        patient_id=patient_object.id,
        created=environment.now,
        status=models.AppointmentStatus.BOOKED,
        practitioner_id=practitioner_object.id,
        duration=appointment_duration,
        scheduled_start_time=scheduled_start_time,
    )

    print(
        f"[{environment.now:>4}] {patient_object.id} scheduled with {practitioner_object.id} at {scheduled_start_time} for {appointment_duration} min"
    )

    total_wait_time = int(scheduled_start_time - environment.now)
    cancellation_check_time = 0
    if total_wait_time > 0:
        cancellation_check_time = random.randint(0, total_wait_time)
        yield environment.timeout(cancellation_check_time)

        if random.random() < APPOINTMENT_CANCEL_PROBABILITY:
            fhir_logger.update_appointment_status(
                appointment_id=appointment_id,
                new_status=models.AppointmentStatus.CANCELLED,
                recorded=environment.now,
                practitioner_id=practitioner_object.id,
                reason="Patient cancelled",
            )
            print(
                f"[{environment.now:>4}] {patient_object.id} CANCELLED appointment with {practitioner_object.id}"
            )
            return None

    remaining_wait_time = total_wait_time - cancellation_check_time
    yield environment.timeout(remaining_wait_time)

    if random.random() < APPOINTMENT_NOSHOW_PROBABILITY:
        fhir_logger.update_appointment_status(
            appointment_id=appointment_id,
            new_status=models.AppointmentStatus.NOSHOW,
            recorded=environment.now,
            practitioner_id=practitioner_object.id,
        )
        print(
            f"[{environment.now:>4}] {patient_object.id} NO-SHOW for appointment with {practitioner_object.id}"
        )
        return None

    with practitioner_object.resource.request() as request:
        yield request
        active_appointments.add(key)

        print(
            f"[{environment.now:>4}] {practitioner_object.id} starts APPOINTMENT with {patient_object.id} ({appointment_duration} min)"
        )

        if random.random() < OBSERVATIONS_DURING_APPOINTMENT_PROBABILITY:
            obs_process = environment.process(
                observations(
                    environment=environment,
                    fhir_logger=fhir_logger,
                    practitioner_object=practitioner_object,
                    patient_object=patient_object,
                    encounter_start=environment.now,
                    remaining_appointment_duration=appointment_duration,
                    encounter_id=None,
                )
            )
            yield obs_process
        else:
            main_process = environment.process(
                encounter(
                    environment=environment,
                    fhir_logger=fhir_logger,
                    practitioner_object=practitioner_object,
                    patient_object=patient_object,
                    appointment_id=appointment_id,
                    appointment_start=scheduled_start_time,
                    appointment_duration=appointment_duration,
                )
            )
            if random.random() < BTG_ACCESS_PROBABILITY:
                btg_proc = environment.process(
                    resource_access_process(
                        environment=environment,
                        fhir_logger=fhir_logger,
                        practitioner_object=practitioner_object,
                        patient_object=patient_object,
                        event_type=models.AccessEventType.EMERGENCY,
                        context_resource_type="Appointment",
                        context_resource_id=appointment_id,
                    )
                )
                yield main_process | btg_proc
            else:
                yield main_process

        active_appointments.discard(key)

        fhir_logger.update_appointment_status(
            appointment_id=appointment_id,
            new_status=models.AppointmentStatus.FINISHED,
            recorded=environment.now,
            practitioner_id=practitioner_object.id,
        )


def encounter(
    environment,
    fhir_logger: models.FHIRLogger,
    practitioner_object: models.Practitioner,
    patient_object: models.Patient,
    appointment_id: str | None,
    appointment_start: int,
    appointment_duration: int,
):
    print(
        f"[{environment.now:>4}] {practitioner_object.id} begins ENCOUNTER with {patient_object.id}"
    )

    encounter_duration = max(
        min(APPOINTMENT_VISIT_DURATIONS),
        random.randint(appointment_duration // 2, appointment_duration),
    )

    max_delay = appointment_duration - encounter_duration
    start_delay = random.randint(0, max_delay)

    encounter_start = appointment_start + start_delay
    encounter_end = encounter_start + encounter_duration
    remaining_appointment_duration = appointment_duration - start_delay

    main_process = environment.timeout(encounter_duration)

    encounter_id = fhir_logger.log_encounter(
        patient_id=patient_object.id,
        actual_start_time=encounter_start,
        duration=encounter_duration,
        practitioner_id=practitioner_object.id,
        appointment_id=appointment_id,
    )

    if random.random() < BTG_ACCESS_PROBABILITY:
        btg_proc = environment.process(
            resource_access_process(
                environment=environment,
                fhir_logger=fhir_logger,
                practitioner_object=practitioner_object,
                patient_object=patient_object,
                event_type=models.AccessEventType.EMERGENCY,
                context_resource_type="Encounter",
                context_resource_id=encounter_id,
            )
        )
        yield main_process | btg_proc
    else:
        yield main_process

    if random.random() < OBSERVATIONS_DURING_ENCOUNTER_PROBABILITY:
        yield environment.process(
            observations(
                environment,
                fhir_logger=fhir_logger,
                practitioner_object=practitioner_object,
                patient_object=patient_object,
                encounter_start=encounter_start,
                remaining_appointment_duration=remaining_appointment_duration,
                encounter_id=encounter_id,
            )
        )


def biased_times(start_time, end_time, count: int, bias_strength: float = 2.0):
    total_minutes = end_time - start_time
    if total_minutes == 0:
        raise ValueError("total_minutes = 0, so no biased times can be generated!")

    points: set[int] = set()
    while len(points) < count:
        r = random.random() ** bias_strength
        minute_offset = int(r * total_minutes)
        points.add(minute_offset)

    return [start_time + offset for offset in sorted(points)]


def observations(
    environment,
    fhir_logger: models.FHIRLogger,
    practitioner_object: models.Practitioner,
    patient_object: models.Patient,
    encounter_start: int,
    remaining_appointment_duration: int,
    encounter_id: str | None,
):
    if remaining_appointment_duration == 0:
        appointment_duration = min(APPOINTMENT_VISIT_DURATIONS)
        encounter_duration = random.randint(
            appointment_duration // 2, appointment_duration
        )
        max_delay = appointment_duration - encounter_duration
        start_delay = random.randint(0, max_delay)
        remaining_appointment_duration = appointment_duration - start_delay
    count = random.randint(1, OBSERVATIONS_MAX)
    obs_times = biased_times(
        encounter_start,
        encounter_start + remaining_appointment_duration,
        count=count,
        bias_strength=1.75,
    )

    for i in range(count):
        code, display = OBSERVATION_CODES[i % OBSERVATIONS_MAX]
        value = (
            f"{random.uniform(96, 99):.1f} °F"
            if i == 0
            else str(random.randint(60, 100))
        )
        print(f"[{obs_times[i]:>4}] Observation {i+1} for patient {patient_object.id}")

        obs_id = fhir_logger.log_observation(
            patient_id=patient_object.id,
            practitioner_id=practitioner_object.id,
            timestamp=obs_times[i],
            code=code,
            value=value,
            encounter_id=encounter_id,
        )

        if random.random() < BTG_ACCESS_PROBABILITY:
            yield environment.process(
                resource_access_process(
                    environment=environment,
                    fhir_logger=fhir_logger,
                    practitioner_object=practitioner_object,
                    patient_object=patient_object,
                    event_type=models.AccessEventType.EMERGENCY,
                    context_resource_type="Observation",
                    context_resource_id=obs_id,
                )
            )

        yield environment.timeout(0)


def resource_access_process(
    environment,
    fhir_logger: models.FHIRLogger,
    practitioner_object: models.Practitioner,
    patient_object: models.Patient,
    event_type: models.AccessEventType,
    context_resource_type: Union[str, None] = None,
    context_resource_id: Union[str, None] = None,
):
    yield environment.timeout(0)

    if event_type == models.AccessEventType.EMERGENCY:
        purpose = models.AccessEventPurpose.EMERGENCY
        if context_resource_type:
            purpose_of_event = f"Emergency access during {context_resource_type}"
        else:
            purpose_of_event = "Emergency access - standalone event"
    elif event_type == models.AccessEventType.CARE:
        purpose = models.AccessEventPurpose.CARE
        if context_resource_type:
            purpose_of_event = f"Normal access during {context_resource_type}"
        else:
            purpose_of_event = "Normal access - standalone event"
    else:
        raise ValueError(
            f"The handling of {event_type} events has not yet been implemented!"
        )

    fhir_logger.log_access_event(
        patient_id=patient_object.id,
        recorded=environment.now,
        practitioner_id=practitioner_object.id,
        action="R",
        event_type=event_type,
        purpose=purpose,
        purpose_of_event=purpose_of_event,
        target_resource_type=context_resource_type,
        target_resource_id=context_resource_id,
        outcome="success",
    )

    print(
        f"[{environment.now:>4}] AUDIT EVENT {event_type}"
        f" by {practitioner_object.id} for {patient_object.id}"
        f"{f' during {context_resource_type}' if context_resource_type else ''}"
    )


def _day_bounds(now_minute: int) -> tuple[int, int]:
    day_start = (now_minute // (24 * 60)) * (24 * 60)
    return day_start, day_start + 24 * 60


def _has_AEO_or_BTG_in_window(
    engine, practitioner_id: str, patient_id: str, start_minute: int, end_minute: int
) -> bool:
    with Session(engine) as session:
        appt = session.exec(
            select(models.Appointment).where(
                (models.Appointment.practitioner_id == practitioner_id)
                & (models.Appointment.patient_id == patient_id)
                & (models.Appointment.scheduled_start_time >= start_minute)
                & (models.Appointment.scheduled_start_time < end_minute)
                & (
                    models.Appointment.status.in_(
                        [
                            models.AppointmentStatus.BOOKED,
                            models.AppointmentStatus.NOSHOW,
                            models.AppointmentStatus.FINISHED,
                        ]
                    )
                )
            )
        ).first()
        if appt:
            return True

        enc = session.exec(
            select(models.Encounter).where(
                (models.Encounter.practitioner_id == practitioner_id)
                & (models.Encounter.patient_id == patient_id)
                & (models.Encounter.actual_start_time >= start_minute)
                & (models.Encounter.actual_start_time < end_minute)
            )
        ).first()
        if enc:
            return True

        obs = session.exec(
            select(models.Observation).where(
                (models.Observation.practitioner_id == practitioner_id)
                & (models.Observation.patient_id == patient_id)
                & (models.Observation.timestamp >= start_minute)
                & (models.Observation.timestamp < end_minute)
            )
        ).first()
        if obs:
            return True

        btg = session.exec(
            select(models.AccessEvent).where(
                (models.AccessEvent.practitioner_id == practitioner_id)
                & (models.AccessEvent.patient_id == patient_id)
                & (models.AccessEvent.recorded >= start_minute)
                & (models.AccessEvent.recorded < end_minute)
                & (models.AccessEvent.event_type == models.AccessEventType.EMERGENCY)
            )
        ).first()
        if btg:
            return True

    return False


def standalone_access_event_generator(
    environment, fhir_logger, practitioner_objects, patient_objects
):
    while True:
        yield environment.timeout(random.randint(60, 60 * 24))

        if random.random() < STANDALONE_BTG_ACCESS_PROBABILITY:
            practitioner_id = random.choice(list(practitioner_objects.keys()))
            patient_id = random.choice(list(patient_objects.keys()))
            practitioner = practitioner_objects[practitioner_id]
            patient = patient_objects[patient_id]

            environment.process(
                resource_access_process(
                    environment=environment,
                    fhir_logger=fhir_logger,
                    practitioner_object=practitioner,
                    patient_object=patient,
                    event_type=models.AccessEventType.EMERGENCY,
                )
            )
        elif random.random() < STANDALONE_NORMAL_ACCESS_PROBABILITY:
            practitioner_id = random.choice(list(practitioner_objects.keys()))
            patient_id = random.choice(list(patient_objects.keys()))
            practitioner = practitioner_objects[practitioner_id]
            patient = patient_objects[patient_id]

            day_start, day_end = _day_bounds(environment.now)
            if not _has_AEO_or_BTG_in_window(
                fhir_logger.engine, practitioner_id, patient_id, day_start, day_end
            ):
                environment.process(
                    resource_access_process(
                        environment=environment,
                        fhir_logger=fhir_logger,
                        practitioner_object=practitioner,
                        patient_object=patient,
                        event_type=models.AccessEventType.CARE,
                    )
                )


def choose_event_type() -> str:
    return random.choices(
        population=list(EVENT_TYPE_WEIGHTS.keys()),
        weights=list(EVENT_TYPE_WEIGHTS.values()),
        k=1,
    )[0]


def patient_process(
    engine,
    environment,
    fhir_logger: models.FHIRLogger,
    practitioner_object: models.Practitioner,
    patient_object: models.Patient,
):
    event_type = choose_event_type()

    if event_type == "Appointment":
        yield environment.process(
            appointment(
                engine,
                environment=environment,
                fhir_logger=fhir_logger,
                practitioner_object=practitioner_object,
                patient_object=patient_object,
            )
        )
    elif event_type == "Encounter":
        encounter_duration = APPOINTMENT_VISIT_DURATION()
        current_time = environment.now

        if is_time_available(
            engine,
            environment,
            practitioner_object,
            current_time,
            duration=encounter_duration,
        ):
            yield environment.process(
                encounter(
                    environment=environment,
                    fhir_logger=fhir_logger,
                    practitioner_object=practitioner_object,
                    patient_object=patient_object,
                    appointment_start=current_time,
                    appointment_duration=encounter_duration,
                    appointment_id=None,
                )
            )
        else:
            print(f"[{current_time:>4}] Could not start encounter - practitioner busy")
    elif event_type == "Observation":
        current_time = environment.now

        if is_time_available(
            engine, environment, practitioner_object, current_time, duration=1
        ):
            yield environment.process(
                observations(
                    environment=environment,
                    fhir_logger=fhir_logger,
                    practitioner_object=practitioner_object,
                    patient_object=patient_object,
                    encounter_start=current_time,
                    remaining_appointment_duration=0,
                    encounter_id=None,
                )
            )
        else:
            print(
                f"[{current_time:>4}] Could not record observation - practitioner busy"
            )


def fill_patient_queues(
    environment,
    patient_queues,
    patient_objects: list,
):
    number_of_practitioners = len(patient_queues)
    count = 0
    while count < len(patient_objects):
        patient_object = patient_objects[count]
        index = count % number_of_practitioners
        key = list(patient_queues.keys())[index]
        queue = patient_queues[key]
        yield queue.put(patient_object)
        print(
            f"[{environment.now}] Patient {patient_object.id} assigned to queue {key}"
        )
        count += 1


def practitioner_process(
    environment,
    fhir_logger,
    practitioner_id,
    practitioner_object,
    patient_queue,
    active_patient_count,
    last_patient_activity,
    patient_objects,
    engine,
):
    while True:
        if active_patient_count.level < PATIENT_MIN_POPULATION or (
            random.random() < PATIENT_ADMITTANCE_PROBABILITY
            and active_patient_count.level < PATIENT_TARGET_POPULATION
        ):
            new_patients_needed = PATIENT_TARGET_POPULATION - active_patient_count.level
            _new_patients = [create_patient(engine) for _ in range(new_patients_needed)]
            new_patients = {patient.id: patient for patient in _new_patients}

            environment.process(
                fill_patient_queues(
                    environment,
                    {practitioner_id: patient_queue},
                    list(new_patients.values()),
                )
            )
            yield active_patient_count.put(new_patients_needed)
            print(
                f"[{environment.now:>4}] Added {new_patients_needed} new patients (Total: {active_patient_count.level})"
            )

        patient_object = yield patient_queue.get()

        current_time = environment.now

        if patient_object.id in last_patient_activity:
            (recorded_time, cooldown) = last_patient_activity[patient_object.id]
            time_since_last = current_time - recorded_time

            if time_since_last < cooldown:
                remaining_cooldown = cooldown - time_since_last
                yield environment.timeout(remaining_cooldown)
                yield patient_queue.put(patient_object)
                continue

        main_process = environment.process(
            patient_process(
                engine, environment, fhir_logger, practitioner_object, patient_object
            )
        )
        yield main_process

        if random.random() < PATIENT_DISCHARGE_PROBABILITY:
            yield active_patient_count.get(1)
            if patient_object.id in last_patient_activity:
                del last_patient_activity[patient_object.id]
            print(
                f"[{environment.now:>4}] Patient {patient_object.id} discharged (Remaining: {active_patient_count.level})"
            )
        else:
            patient_queue.put(patient_object)
            print(
                f"[{environment.now:>4}] Patient {patient_object.id} re-queued (eligible after cooldown)"
            )

            last_patient_activity[patient_object.id] = (
                environment.now,
                PATIENT_SCHEDULING_COOLDOWN_IN_MINUTES(),
            )


def scheduler(
    engine,
    environment,
    fhir_logger,
    patient_queues,
    practitioner_objects,
    patient_objects,
):
    active_patient_count = simpy.Container(
        environment, init=len(patient_objects), capacity=PATIENT_TARGET_POPULATION
    )
    last_patient_activity = {}

    print(f"[{environment.now:>4}] Starting with {active_patient_count.level} patients")

    environment.process(
        fill_patient_queues(environment, patient_queues, list(patient_objects.values()))
    )

    practitioner_processes = [
        environment.process(
            practitioner_process(
                environment,
                fhir_logger,
                practitioner_id,
                practitioner_object,
                patient_queues[practitioner_id],
                active_patient_count,
                last_patient_activity,
                patient_objects,
                engine,
            )
        )
        for practitioner_id, practitioner_object in practitioner_objects.items()
    ]

    yield simpy.events.AllOf(environment, practitioner_processes)


def create_patient(engine):
    patient = models.Patient(
        id=fake.uuid4(),
        first_name=fake.first_name(),
        last_name=fake.last_name(),
        gender=fake.random_element(elements=["male", "female"]),
        birthdate=fake.date_of_birth(),
    )
    with Session(engine) as session:
        session.add(patient)
        session.commit()
        session.refresh(patient)
    return patient


def create_practitioner(engine, environment, role="doctor"):
    if role == "doctor":
        practitioner = models.Practitioner(
            env=environment,
            work_schedule=utilities.sample_practitioner_work_schedule(),
            id=fake.uuid4(),
            first_name=fake.first_name(),
            last_name=fake.last_name(),
            gender=fake.random_element(elements=["male", "female"]),
            birthdate=fake.date_of_birth(),
            role=role,
        )
        with Session(engine) as session:
            session.add(practitioner)
            session.commit()
            session.refresh(practitioner)
        return practitioner
    else:
        raise ValueError("Other roles than 'doctor' is currently not supported.")


def run_simulation(engine, pracitioners: int, patients: int):
    environment = simpy.Environment()

    _patient_objects = [create_patient(engine) for _ in range(patients)]
    patient_objects = {patient.id: patient for patient in _patient_objects}

    _practitioner_objects = [
        create_practitioner(engine, environment) for _ in range(pracitioners)
    ]
    practitioner_objects = {
        practitioner.id: practitioner for practitioner in _practitioner_objects
    }
    patient_queues = {
        practitioner_object.id: simpy.Store(environment)
        for practitioner_object in _practitioner_objects
    }

    provenance_tracker = models.ProvenanceTracker(engine=engine)

    fhir_logger = models.FHIRLogger(
        engine=engine, provenance_tracker=provenance_tracker
    )

    environment.process(
        scheduler(
            engine=engine,
            environment=environment,
            fhir_logger=fhir_logger,
            patient_queues=patient_queues,
            practitioner_objects=practitioner_objects,
            patient_objects=patient_objects,
        )
    )

    environment.process(
        standalone_access_event_generator(
            environment, fhir_logger, practitioner_objects, patient_objects
        )
    )

    environment.run(until=SIMULATION_DURATION_IN_MINUTES)


def main():
    db_filename = "hospital_simulation.db"

    if os.path.exists(db_filename):
        os.remove(db_filename)
    engine = create_engine(f"sqlite:///{db_filename}")

    SQLModel.metadata.create_all(engine)

    print(f"STARTING SIMULATION OF DURATION: {int(SIMULATION_DURATION_IN_MINUTES):>4}")

    run_simulation(
        engine=engine,
        pracitioners=NUMBER_OF_PRACTITIONERS,
        patients=NUMBER_OF_PATIENTS,
    )
    print(f"[{int(SIMULATION_DURATION_IN_MINUTES):>4}] REACHED END OF SIMULATION.")


if __name__ == "__main__":
    main()




In [ ]:
!pip install sqlmodel


In [ ]:
pip install --upgrade pip

In [ ]:
pip install simpy


In [ ]:
!pip install Faker
